In [39]:
import utils as u
import preprocessing as pp
import train as tt
import visualization as viz

import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np

from keras.utils import np_utils

import spectral
from spectral import spy_colors

#tests
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score

In [49]:
numPCAcomponents = 100
PATCH_SIZE = 5
C1 = 3*numPCAcomponents

batch_size = 128
epochs = 10

classes_authorized = [2,3,5,6,10,0,11,12,14,15]

target_names = ['Maïs- Pas de Technique de Conservation des sols',
                    'Maïs- Minimum Tillage',
                    'Herbe-Pâturage',
                    'Herbe-Arbre',
                    'Soja-Pas de Technique de Conservation des sols', 'Soja-Minimum Tillage','Soja', 'Bois',
                    'Bâtiment-Herbe-Arbre-drives',]

label_dictionary = {
    0: 'Rien',
    1: 'Maïs- Pas de Technique de Conservation des sols', 
    2: 'Maïs- Minimum Tillage',
    3: 'Herbe-Pâturage', 
    4: 'Herbe-Arbre', 
    5: 'Soja-Pas de Technique de Conservation des sols', 
    6: 'Soja-Minimum Tillage',
    7: 'Soja', 
    8: 'Bois', 
    9: 'Bâtiment-Herbe-Arbre-drives',
}

In [50]:
X, train_data, test_data = pp.loadData()

In [51]:
train_data = pp.deleteUselessClasses(train_data, classes_authorized)
test_data = pp.deleteUselessClasses(test_data, classes_authorized)

def pretty_print_count(train_data, test_data):
    unique_train = np.unique(train_data, return_counts=True)
    unique_test = np.unique(test_data, return_counts=True)
    for i in range(0, unique_train[0].shape[0]):
        print(unique_train[0][i], "=>", unique_train[1][i], "/", unique_test[1][i])
    print()
        
print("Before Shuffle: ")
pretty_print_count(train_data, test_data)
train_data, test_data = pp.shuffleTrainTest(train_data, test_data)
print("After Shuffle: ")
pretty_print_count(train_data, test_data)

Before Shuffle: 
0 => 15155 / 17753
1 => 648 / 780
2 => 751 / 79
3 => 465 / 18
4 => 492 / 238
5 => 231 / 741
6 => 1646 / 809
7 => 403 / 190
8 => 1145 / 120
9 => 89 / 297

After Shuffle: 
0 => 16441 / 16467
1 => 700 / 728
2 => 435 / 395
3 => 249 / 234
4 => 382 / 348
5 => 481 / 491
6 => 1205 / 1250
7 => 317 / 276
8 => 635 / 630
9 => 180 / 206



In [52]:
X, pca = pp.dimensionalityReduction(X, numComponents=numPCAcomponents, standardize=False)

In [53]:
X_train, X_test, y_train, y_test = pp.split(X, train_data, test_data)

In [54]:
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [ ]:
y_pred = neigh.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

In [34]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [ ]:
X_train2 = (X_train - np.min(X_train))
X_test2 = (X_test - np.min(X_test))
multi = MultinomialNB(alpha=0.001)
multi.fit(X_train2, y_train)

In [ ]:
y_pred = multi.predict(X_test2)

In [ ]:
set(y_test) - set(y_pred)
print(classification_report(y_test, y_pred, target_names=target_names))

In [16]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier(max_depth=10, criterion="entropy", max_leaf_nodes=50)

In [ ]:
dtree.fit(X_train, y_train)

In [ ]:
y_pred = dtree.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
svc = SVC(C=9, decision_function_shape="ovo")

In [ ]:
svc.fit(X_train, y_train)

In [ ]:
y_pred = svc.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=target_names))

In [48]:

    
train_test_model(dtree, X_train, y_train, X_test, y_test)

Model f1_score on train data/labels: 0.7402883872794087 

Model score on test data/labels:
                                                 precision    recall  f1-score   support

Maïs- Pas de Technique de Conservation des sols       0.69      0.57      0.63       728
                          Maïs- Minimum Tillage       0.62      0.40      0.49       395
                                 Herbe-Pâturage       0.80      0.74      0.77       234
                                    Herbe-Arbre       0.86      0.90      0.88       348
 Soja-Pas de Technique de Conservation des sols       0.62      0.67      0.64       491
                           Soja-Minimum Tillage       0.66      0.77      0.71      1250
                                           Soja       0.50      0.52      0.51       276
                                           Bois       0.88      0.91      0.90       630
                    Bâtiment-Herbe-Arbre-drives       0.54      0.48      0.51       206

                 